## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [1]:
# already set up on Expanse; toggle for colab

# pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch] pyfaidx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 32.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.2 MB/s eta 0:00:00


Set path prefix for Expanse

In [ ]:
path_prefix = "/home/sgriesmer/"

Import SNP datasets

In [3]:
import pandas as pd

snp_lt_05 = pd.read_csv(path_prefix + "DNABERT/Datasets/CAD/C1_SNPlt05.vcf", sep='\t')
snp_gt_5 = pd.read_csv(path_prefix + "DNABERT/Datasets/CAD/C1_SNPgt5.vcf", sep='\t')
snp_lt_05.head(), snp_lt_05.shape, snp_gt_5.head(), snp_gt_5.shape



(         Name  Chromosome  Position Reference Alternative
 0  rs62639963           1    889403         T           A
 1  rs13303278           1    921716         C           A
 2  rs34712273           1    924528         C           A
 3   rs3128111           1    930751         G           C
 4   rs2799060           1    931362         A           G,
 (63541, 5),
           Name  Chromosome  Position Reference Alternative
 0  rs561255355           1    569406         A           G
 1  rs143225517           1    751756         T           C
 2    rs3115860           1    753405         A           C
 3    rs2073813           1    753541         G           A
 4    rs3131969           1    754182         G           A,
 (985034, 5))

Choose test set for run

In [4]:
snp_test = snp_lt_05
snp_test_name = "SNPlt05"
#snp_test = snp_gt_5
#snp_test_name = "SNPgt5"

Import reference genome

In [5]:
from pyfaidx import Fasta

ref_genome = Fasta(path_prefix + "selene/selene_quickstart_tutorial/male.hg19.fasta")
ref_genome["chr1"]

FastaRecord("chr1")

Generate reference and alternative sequences from alleles and save as csv file.

In [6]:
# create a dataset

column_names = ["names", "ref_seq", "alt_seq"]
snp_seq_dataset = pd.DataFrame(columns=column_names)

# create reference and alternate sequences

seq_len = 50
for i,snp in enumerate(snp_test["Name"][0:5000]):
  chrom = "chr" + str(snp_test["Chromosome"][i])
  pos = snp_test["Position"][i]
  ref_allele = snp_test["Reference"][i]
  alt_allele = snp_test["Alternative"][i]
  ref_sequence = ref_genome[chrom][pos-seq_len-1:pos+seq_len].seq
  ref_sequence = ref_sequence[0:seq_len] + ref_allele + ref_sequence[seq_len+len(ref_allele):]
  alt_sequence = ref_sequence[0:seq_len] + alt_allele + ref_sequence[seq_len+len(alt_allele):]

  # make uppercase

  ref_sequence = ref_sequence.upper()
  alt_sequence = alt_sequence.upper()

  # write into dataset

  snp_seq_dataset.loc[i] = [snp, ref_sequence, alt_sequence]

In [7]:
snp_seq_dataset

,names,ref_seq,alt_seq
0,rs62639963,GGCCCCCGTGCCCTCCCCACCAGAATAGCACCTGTATGGCCGAGCC...,GGCCCCCGTGCCCTCCCCACCAGAATAGCACCTGTATGGCCGAGCC...
1,rs13303278,TTCACCGTGTTAGCCAGGATGGTCTCGAACTCCTGACCTCGTGATC...,TTCACCGTGTTAGCCAGGATGGTCTCGAACTCCTGACCTCGTGATC...
2,rs34712273,GTTTCTGGAGTTCTGGGTTGACTGTTTCTGGAGTTCAGGGTTGATT...,GTTTCTGGAGTTCTGGGTTGACTGTTTCTGGAGTTCAGGGTTGATT...
3,rs3128111,CAGCCTGGGCGACAGAGACTCTGTCTAAACAAAACAAAAGAAAACC...,CAGCCTGGGCGACAGAGACTCTGTCTAAACAAAACAAAAGAAAACC...
4,rs2799060,GCGGTTGTGTTGCCCCGTTCCCCCCGCGGCAGACAAGCCCAGACAC...,GCGGTTGTGTTGCCCCGTTCCCCCCGCGGCAGACAAGCCCAGACAC...
...,...,...,...
4995,rs61424628,TTGTATTCACCAAGACCACACTAAGCATGCACTCTGGTGATCTGTA...,TTGTATTCACCAAGACCACACTAAGCATGCACTCTGGTGATCTGTA...
4996,rs183155157,AAATAGGAATTAAGAGTCAGCAACAGATTAATAATTTTTTAAAATT...,AAATAGGAATTAAGAGTCAGCAACAGATTAATAATTTTTTAAAATT...
4997,rs78156697,GAAGCACCACCAGCCCCACTACAGCCTCCGCTACCACCGCTTCTCT...,GAAGCACCACCAGCCCCACTACAGCCTCCGCTACCACCGCTTCTCT...
4998,rs113441334,ATCACTTGAACCTGGGAGGTGGAGGTTGCAGTGAGCCAAGATAGCG...,ATCACTTGAACCTGGGAGGTGGAGGTTGCAGTGAGCCAAGATAGCG...


Transform into Hugging Face Dataset for prediction

In [8]:
from datasets import Dataset, DatasetDict, load_metric

Dataset_snp_seq = Dataset.from_pandas(snp_seq_dataset)

In [9]:
Dataset_snp_seq

Dataset({
    features: ['names', 'ref_seq', 'alt_seq', '__index_level_0__'],
    num_rows: 5000
})

Generate predictions on each TFBS feature for reference and alterative sequences

Make dataset to store predictions for all TFBS feature models

In [10]:
# create datasets

column_names = ["TFBS dataset"]
prob_predictions_ref_dataset = pd.DataFrame(columns=column_names)
prob_predictions_ref_dataset["TFBS dataset"] = Dataset_snp_seq["names"]
prob_predictions_alt_dataset = pd.DataFrame(columns=column_names)
prob_predictions_alt_dataset["TFBS dataset"] = Dataset_snp_seq["names"]
prob_predictions_diff_dataset = pd.DataFrame(columns=column_names)
prob_predictions_diff_dataset["TFBS dataset"] = Dataset_snp_seq["names"]
prob_predictions_odds_dataset = pd.DataFrame(columns=column_names)
prob_predictions_odds_dataset["TFBS dataset"] = Dataset_snp_seq["names"]


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
import numpy as np
import sys
import os

# initialize parameters

for fname in ["BroadDnd41CtcfUniPk-ran.csv",
"BroadDnd41Ezh239875UniPk-ran.csv",
"BroadGm12878CtcfUniPk-ran.csv"]:

  dsname = path_prefix + "DNABERT/Datasets/tfbs/" + fname
  tfbs_dataset = pd.read_csv(dsname, sep=',')

# tokenization

  def kmers_stride1(s, k=6):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

# load pre-trained model

  model_path = path_prefix + "DNABERT/Output_Models/" + fname.split(".")[0] + "/"
  model_cls = AutoModelForSequenceClassification.from_pretrained(model_path)
  tokenizer = AutoTokenizer.from_pretrained(model_path)

# reformat data to Hugging Face Dataset format from pandas

  def tok_func_ref(x): return tokenizer(" ".join(kmers_stride1(x["ref_seq"])))
  def tok_func_alt(x): return tokenizer(" ".join(kmers_stride1(x["alt_seq"])))

  Dataset_snp_seq_tok_ref = Dataset_snp_seq.map(tok_func_ref, batched=False)
  Dataset_snp_seq_tok_alt = Dataset_snp_seq.map(tok_func_alt, batched=False)

  dds = DatasetDict({
    'eval_ref': Dataset_snp_seq_tok_ref,
    'eval_alt': Dataset_snp_seq_tok_alt
  })

# switch to GPU

  if torch.cuda.device_count() > 0:
    model_cls.to('cuda')

# load model

  bs = 32
  epochs = 4
  lr = 8e-5

  args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

  def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# predictions from reference alleles

  trainer = Trainer(model_cls, args, eval_dataset=dds['eval_ref'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

  eval_preds_ref = trainer.predict(dds['eval_ref'])

# predictions from alternative alleles

  trainer = Trainer(model_cls, args, eval_dataset=dds['eval_alt'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

  eval_preds_alt = trainer.predict(dds['eval_alt'])

# find reference probabilities

  from scipy.special import softmax

  prob_predictions_ref = softmax(eval_preds_ref[0], axis=1)
  pos_prob_predictions_ref = prob_predictions_ref[:, 1]

# find alternative probabilities

  prob_predictions_alt = softmax(eval_preds_alt[0], axis=1)
  pos_prob_predictions_alt = prob_predictions_alt[:, 1]

# print probabilities into dataset

  prob_predictions_ref_dataset[fname.split(".")[0]] = pos_prob_predictions_ref

  prob_predictions_alt_dataset[fname.split(".")[0]] = pos_prob_predictions_alt

print(prob_predictions_ref_dataset)

print(prob_predictions_alt_dataset)





Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


     TFBS dataset  BroadDnd41CtcfUniPk  BroadDnd41Ezh239875UniPk  \
0      rs62639963             0.062674                  0.772574   
1      rs13303278             0.999596                  0.822260   
2      rs34712273             0.004755                  0.177954   
3       rs3128111             0.998266                  0.744981   
4       rs2799060             0.985029                  0.916409   
...           ...                  ...                       ...   
4995   rs61424628             0.999509                  0.951097   
4996  rs183155157             0.002924                  0.635647   
4997   rs78156697             0.985773                  0.575222   
4998  rs113441334             0.999649                  0.813238   
4999  rs111234954             0.999668                  0.886813   

      BroadGm12878CtcfUniPk  
0                  0.045353  
1                  0.999711  
2                  0.030186  
3                  0.999791  
4                  0.999469  
...

Drop SNP names from datasets to subtract them

In [12]:
snp_names = prob_predictions_ref_dataset["TFBS dataset"]

In [13]:
prob_predictions_ref_dataset_nosnp = prob_predictions_ref_dataset.drop("TFBS dataset", axis=1)
prob_predictions_alt_dataset_nosnp = prob_predictions_alt_dataset.drop("TFBS dataset", axis=1)

Find the differences between the two sets and take the absolute value

In [14]:
prob_predictions_diff_dataset_nosnp = prob_predictions_ref_dataset_nosnp - prob_predictions_alt_dataset_nosnp
prob_predictions_diff_dataset_nosnp

,BroadDnd41CtcfUniPk,BroadDnd41Ezh239875UniPk,BroadGm12878CtcfUniPk
0,-0.055354,0.020607,-1.502607e-01
1,-0.000024,0.001432,1.031160e-05
2,0.000424,0.001282,5.150560e-03
3,0.002009,-0.044658,8.344650e-07
4,0.036444,0.040888,1.387000e-04
...,...,...,...
4995,0.000000,0.000000,0.000000e+00
4996,0.000503,0.034960,1.409649e-02
4997,-0.002796,-0.022503,-1.408339e-03
4998,-0.000011,-0.004336,-4.410744e-06


In [15]:
prob_predictions_absdiff_dataset_nosnp = abs(prob_predictions_ref_dataset_nosnp - prob_predictions_alt_dataset_nosnp)
prob_predictions_absdiff_dataset_nosnp

,BroadDnd41CtcfUniPk,BroadDnd41Ezh239875UniPk,BroadGm12878CtcfUniPk
0,0.055354,0.020607,1.502607e-01
1,0.000024,0.001432,1.031160e-05
2,0.000424,0.001282,5.150560e-03
3,0.002009,0.044658,8.344650e-07
4,0.036444,0.040888,1.387000e-04
...,...,...,...
4995,0.000000,0.000000,0.000000e+00
4996,0.000503,0.034960,1.409649e-02
4997,0.002796,0.022503,1.408339e-03
4998,0.000011,0.004336,4.410744e-06



Find max absolute difference in probabilities across TFs

In [16]:
max_absdiff_prob_predictions = prob_predictions_absdiff_dataset_nosnp.max(axis=1)
max_absdiff_prob_predictions_df = pd.DataFrame(columns=["snp", "max_absdiff_prob"])
max_absdiff_prob_predictions_df["snp"] = snp_names
max_absdiff_prob_predictions_df["max_absdiff_prob"] = max_absdiff_prob_predictions
max_absdiff_prob_predictions_df




,snp,max_absdiff_prob
0,rs62639963,0.150261
1,rs13303278,0.001432
2,rs34712273,0.005151
3,rs3128111,0.044658
4,rs2799060,0.040888
...,...,...
4995,rs61424628,0.000000
4996,rs183155157,0.034960
4997,rs78156697,0.022503
4998,rs113441334,0.004336


In [17]:
max_absdiff_prob_predictions_df["max_absdiff_prob"].describe()

count    5000.000000
mean        0.103362
std         0.177130
min         0.000000
25%         0.007293
50%         0.029603
75%         0.107780
max         0.981654
Name: max_absdiff_prob, dtype: float64

Write results to output file

In [18]:
output_file = path_prefix + "DNABERT/output/" + snp_test_name + "-results_by_variant.csv"
max_absdiff_prob_predictions_df.to_csv(output_file, index=False, sep='\t')


Find difference between the log odds scores of alleles

In [19]:
prob_predictions_logodds_dataset_nosnp = (np.log2(prob_predictions_ref_dataset_nosnp/(1-prob_predictions_ref_dataset_nosnp))
- np.log2(prob_predictions_alt_dataset_nosnp/(1-prob_predictions_alt_dataset_nosnp)))
prob_predictions_logodds_dataset_nosnp




,BroadDnd41CtcfUniPk,BroadDnd41Ezh239875UniPk,BroadGm12878CtcfUniPk
0,-1.001011,0.164135,-2.355826
1,-0.089732,0.014089,0.050569
2,0.135252,0.012680,0.277550
3,1.113038,-0.361730,0.005749
4,1.834364,0.640338,0.335261
...,...,...,...
4995,0.000000,0.000000,0.000000
4996,0.273322,0.213798,1.189096
4997,-0.319817,-0.133890,-2.234329
4998,-0.044839,-0.041562,-0.028431
